In [6]:
# Import necessary libraries
import dspy
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI language model
cheap_lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="low")
smart_lm = dspy.LM("openai/gpt-5", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="high")

print("Cheap LM response:")
print(cheap_lm("Tell me a funny joke"))
print("\nSmart LM response:")
print(smart_lm("Tell me a funny joke"))

Cheap LM response:
['Why did the scarecrow win an award?\n\nBecause he was outstanding in his field.']

Smart LM response:
['I told my therapist I keep hearing a voice telling me to eat candy bars.\nShe said, “Snickers?”\nI said, “No, seriously.”']
